<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Imports" data-toc-modified-id="Imports-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>Imports</a></span></li><li><span><a href="#Logging" data-toc-modified-id="Logging-0.2"><span class="toc-item-num">0.2&nbsp;&nbsp;</span>Logging</a></span></li><li><span><a href="#Generic-Script-code" data-toc-modified-id="Generic-Script-code-0.3"><span class="toc-item-num">0.3&nbsp;&nbsp;</span>Generic Script code</a></span></li><li><span><a href="#Get-soup-object-for-problem-URL" data-toc-modified-id="Get-soup-object-for-problem-URL-0.4"><span class="toc-item-num">0.4&nbsp;&nbsp;</span>Get soup object for problem URL</a></span></li></ul></li><li><span><a href="#Problem-Code" data-toc-modified-id="Problem-Code-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Problem Code</a></span></li><li><span><a href="#Add-checks-to-check-string" data-toc-modified-id="Add-checks-to-check-string-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Add checks to check string</a></span></li></ul></div>

#### Imports

In [3]:
import logging
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import pandas as pd
import re

#### Logging

In [4]:
logging.basicConfig(level=logging.DEBUG)

#### Generic Script code

In [5]:
def _get_response(url):
    response = requests.get(url)
    return response

In [6]:
def get_soup_object(url):
    response = _get_response(url)
    #Insert logic to check
    #if response = 200
    soup = BeautifulSoup(response.text, "html.parser")
    #else handle exceptions
    return soup

In [7]:
def _read_table(table_number, soup):
    table = soup.find_all('table')[table_number-1]
    rows = table.find_all('tr')
    row_list = list()
    for tr in rows:
        td = tr.find_all('td')
        row = [i.text for i in td]
        row_list.append(row)
    flat_list = [item for sublist in row_list for item in sublist]
    return flat_list

In [18]:
def check_pattern_existence(pattern, string):
    if re.search(pattern, string):
        next_line_indicator = True
    else:
        next_line_indicator = False
    
    return next_line_indicator

#### Get soup object for problem URL

In [23]:
debug_url = 'http://cmano-db.com/aircraft/3953/'#3040/'#3206/'#3953/'

In [24]:
soup = get_soup_object(debug_url)

DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): cmano-db.com:80
DEBUG:urllib3.connectionpool:http://cmano-db.com:80 "GET /aircraft/3953/ HTTP/1.1" 200 None


### Problem Code

In [29]:
table_number = 2

flat_list = _read_table(table_number, soup)

_list = []
dict_keys = []
dict_values = []

logging.debug(f"Length of flat_list is: {len(flat_list)}.")
logging.debug(f"flat_list is: {flat_list}.")

next_line_pattern = '\\n[\\t]+'
max_range_pattern = ' Max Range: '

DEBUG:root:Length of flat_list is: 1.
DEBUG:root:flat_list is: ['450 liter Drop Tank -\n\t\t\t\t Drop Tank\n'].


In [30]:
for i in range(len(flat_list)):
    _list_val = flat_list[i]
    if check_pattern_existence(max_range_pattern, _list_val) and check_pattern_existence(next_line_pattern, _list_val):
        _sublist_2 = _list_val.split(max_range_pattern)
        _sublist = re.split(next_line_pattern, _sublist_2[0])
        _sublist.append(_sublist_2[1]) #Specific to this context
        _list.append(_sublist)
    else:
        _list.append([_list_val])
print(_list)

[['450 liter Drop Tank -\n\t\t\t\t Drop Tank\n']]


In [31]:
for i in range(len(_list)):
    dict_keys_template = [f'Sensor {i+1} Name', f'Sensor {i+1} Type', f'Sensor {i+1} Max Range']
    for j in range(len(_list[i])):
        dict_keys.append(dict_keys_template[j])
        dict_values.append(_list[i][j])

In [32]:
_dict = dict(zip(dict_keys, dict_values))

In [33]:
df_table_2 = pd.DataFrame().append(_dict, ignore_index=True)

In [34]:
df_table_2

,Sensor 1 Name
0,450 liter Drop Tank -\n\t\t\t\t Drop Tank\n


### Add checks to check string

In [66]:
flat_list

['OLS-30 [IRST] - (Izdeliye 52Sh) Infrared\n\t\t\t\t\tIRST, Imaging Infrared Seach and Track Max Range: 185.2 km',
 'Tarang Mk.2 - (India, Su-30MKI) ESM\n\t\t\t\t\tRWR, Radar Warning Receiver Max Range: 222.2 km',
 'Slot Back [N-011M Bars Mk3] - (Zhuk-27, Su-30MKI) Radar\n\t\t\t\t\tRadar, FCR, Air-to-Air & Air-to-Surface, Long-Range Max Range: 296.3 km',
 'OLS-30 [Laser Rangefinder] - (Zhuk-27, Su-30MKI) Laser Rangefinder\n\t\t\t\t\tLaser Rangefinder for Weapon Director Max Range: 0 km',
 'SAP-518 - (Russia, 2010+) ECM\n\t\t\t\t\tOECM, Offensive ECM Max Range: 0 km']

In [68]:
sub_str = flat_list[0]
sub_str

'OLS-30 [IRST] - (Izdeliye 52Sh) Infrared\n\t\t\t\t\tIRST, Imaging Infrared Seach and Track Max Range: 185.2 km'

In [70]:
#check if max range parameter exists:
if  in sub_str:
    print("Yes")
else:
    print("No")

Yes


In [75]:
next_line_pattern = '\\n[\\t]+'
max_range_pattern = ' Max Range: '

In [ ]:
def check_pattern_existence(pattern, string):
    if re.search(pattern, string):
        next_line_indicator = True
    else:
        next_line_indicator = False
    
    return next_line_indicator